In [33]:
import pandas as pd
import json 
import requests
import os
from dotenv import load_dotenv

In [34]:
load_dotenv()

True

In [35]:
API_KEY = os.getenv("API_KEY")
CHAIN_ID = "1"  

In [36]:
with open("wallet_address.txt", "r") as file:
    wallet_addresses = [line.strip() for line in file.readlines()]

# Get data token holdings
def get_wallet_holdings(wallet_address):
    url = f"https://api.covalenthq.com/v1/{CHAIN_ID}/address/{wallet_address}/balances_v2/"
    params = {"key": API_KEY}
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error while query {wallet_address}: {response.status_code}")
        return None



In [37]:
# wallet_data=[]

# for wallet in wallet_addresses:
#     data=get_wallet_holdings(wallet)
#     # print(json.dumps(data,indent=4))
#     wallet_data.append(data)

# with open("wallet_data.json","w") as f:
#     json.dump(wallet_data,f,indent=4)


In [38]:
with open("wallet_data.json","r") as f:
    wallet_data=json.load(f)

In [ ]:
token_holdings = {}
total_value = 0.0

for wallet in wallet_data:
    for item in wallet['data']['items']:
        token_symbol = item['contract_ticker_symbol']
        token_value = item['quote'] if item['quote'] is not None else 0.0  
        
        if token_symbol in token_holdings:
            token_holdings[token_symbol] += token_value
        else:
            token_holdings[token_symbol] = token_value
            
        total_value += token_value  

print(token_holdings)


{'OL': 2300.3120228117, 'ETH': 29681.876610832012, 'BIGTIME': 0.0, 'USDC': 8753.049011654, 'WETH': 11855.438350999999, 'USD0 [www.usual.finance]': 0.0, 'BEAM': 5229.58735700058, 'MAVIA': 3.5124998, 'GF': 0.0076374, 'blvtUSDC': 0.0, 'WAGMIGAMES': 1034.7145, 'WAVAX': 833.1505, 'PRIME': 801.0212, 'SFUND': 747.9417, 'DIA': 634.6812, 'POND': 511.17126, 'IMX': 438.7854, 'ATH': 277.9772, 'BLANK': 20.702888400000003, 'XSGD': 0.32179713, 'WBTC': 0.0, 'VIRTUAL': 0.0, 'KERMIT': 0.0, 'MC': 0.0, 'USDT': 949.400211, None: 0.0, 'PETER': 0.0, 'PATH': 0.0, 'SIGMA': 0.0, 'SMCUNILP': 0.0, 'SMC': 0.0, 'Get $VIRTUAL Airdrop at https://virtuals-labs.com': 0.0, 'Buddy': 0.0, 'HEART': 0.0, 'FCOB': 0.0, 'UNI-V2': 0.0, 'Kishui.org': 0.0, 'sMCUNILPV2': 0.0, 'EMC': 0.0, 'TOMI': 0.0086356, 'SIPHER': 37685.29566840004, 'BLOOD': 0.0, 'RWASTE': 0.0, 'LMR': 0.0, 'APE': 0.0, '1CAT': 0.0, 'ZRO': 0.0, 'PEAQ': 0.0, 'ANLOGPS': 0.0, 'sSIPHER': 0.0, 'eSIPHER': 0.0, 'BANK': 5.9244402, 'STRK': 0.0066070664, 'NOOB': 0.0, 'BAT':

In [43]:
len(token_holdings.values())

699